In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Compose
import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
myTransforms=Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=1),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],std=[0.5,0.5,0.5]),
])


In [4]:
cifartrain=datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=myTransforms
)

cifartest=datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=myTransforms
)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_dataloader=DataLoader(dataset=cifartrain,batch_size=64,shuffle=True,num_workers=6,prefetch_factor=6,pin_memory=True,persistent_workers=True)
test_dataloader=DataLoader(dataset=cifartest,batch_size=64,shuffle=True,num_workers=6,prefetch_factor=6,pin_memory=True,persistent_workers=True)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = nn.Softmax(dim=1)(model(X.cuda()))
        loss = loss_fn(pred, y.cuda())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X.cuda())
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = nn.Softmax(dim=1)(model(X.cuda()))
            test_loss += loss_fn(pred, y.cuda()).item()
            correct += (pred.argmax(1) == y.cuda()).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [9]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
batch_size = 64
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303643  [    0/50000]
loss: 2.303249  [ 6400/50000]
loss: 2.301578  [12800/50000]
loss: 2.299190  [19200/50000]
loss: 2.298746  [25600/50000]
loss: 2.299728  [32000/50000]
loss: 2.293784  [38400/50000]
loss: 2.292598  [44800/50000]
Test Error: 
 Accuracy: 22.7%, Avg loss: 2.294300 

Epoch 2
-------------------------------
loss: 2.293275  [    0/50000]
loss: 2.289141  [ 6400/50000]
loss: 2.291543  [12800/50000]
loss: 2.290916  [19200/50000]
loss: 2.283099  [25600/50000]
loss: 2.282347  [32000/50000]
loss: 2.262152  [38400/50000]
loss: 2.274446  [44800/50000]
Test Error: 
 Accuracy: 21.1%, Avg loss: 2.266726 

Epoch 3
-------------------------------
loss: 2.265566  [    0/50000]
loss: 2.242507  [ 6400/50000]
loss: 2.261005  [12800/50000]
loss: 2.260902  [19200/50000]
loss: 2.251687  [25600/50000]
loss: 2.248219  [32000/50000]
loss: 2.232132  [38400/50000]
loss: 2.250453  [44800/50000]
Test Error: 
 Accuracy: 23.2%, Avg loss: 2.222099 

Epoc

In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.142179  [    0/50000]
loss: 2.081818  [ 6400/50000]
loss: 2.082745  [12800/50000]
loss: 2.033105  [19200/50000]
loss: 2.143479  [25600/50000]
loss: 2.089624  [32000/50000]
loss: 2.131165  [38400/50000]
loss: 2.155652  [44800/50000]
Test Error: 
 Accuracy: 36.6%, Avg loss: 2.097827 

Epoch 2
-------------------------------
loss: 2.049086  [    0/50000]
loss: 2.031300  [ 6400/50000]
loss: 2.100774  [12800/50000]
loss: 2.050779  [19200/50000]
loss: 2.161657  [25600/50000]
loss: 2.139183  [32000/50000]
loss: 2.105450  [38400/50000]
loss: 2.116628  [44800/50000]
Test Error: 
 Accuracy: 37.1%, Avg loss: 2.090926 

Epoch 3
-------------------------------
loss: 2.167558  [    0/50000]
loss: 2.106825  [ 6400/50000]
loss: 2.126768  [12800/50000]
loss: 2.161480  [19200/50000]
loss: 2.122772  [25600/50000]
loss: 2.054239  [32000/50000]
loss: 2.150550  [38400/50000]
loss: 2.169883  [44800/50000]
Test Error: 
 Accuracy: 38.1%, Avg loss: 2.082340 

Epoc

In [11]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.014812  [    0/50000]
loss: 1.934906  [ 6400/50000]
loss: 2.057323  [12800/50000]
loss: 2.001268  [19200/50000]
loss: 2.086144  [25600/50000]
loss: 2.135471  [32000/50000]
loss: 2.027562  [38400/50000]
loss: 2.032734  [44800/50000]
Test Error: 
 Accuracy: 41.3%, Avg loss: 2.048391 

Epoch 2
-------------------------------
loss: 2.001186  [    0/50000]
loss: 2.158391  [ 6400/50000]
loss: 1.976650  [12800/50000]
loss: 2.083311  [19200/50000]
loss: 2.073504  [25600/50000]
loss: 2.031892  [32000/50000]
loss: 2.066042  [38400/50000]
loss: 2.106697  [44800/50000]
Test Error: 
 Accuracy: 41.7%, Avg loss: 2.044181 

Epoch 3
-------------------------------
loss: 2.067180  [    0/50000]
loss: 2.053866  [ 6400/50000]
loss: 2.092355  [12800/50000]
loss: 2.002193  [19200/50000]
loss: 2.073191  [25600/50000]
loss: 1.994059  [32000/50000]
loss: 2.121749  [38400/50000]
loss: 2.014260  [44800/50000]
Test Error: 
 Accuracy: 41.9%, Avg loss: 2.040603 

Epoc

In [12]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.005113  [    0/50000]
loss: 2.004045  [ 6400/50000]
loss: 1.990114  [12800/50000]
loss: 2.037858  [19200/50000]
loss: 2.017127  [25600/50000]
loss: 1.973101  [32000/50000]
loss: 1.921767  [38400/50000]
loss: 1.953205  [44800/50000]
Test Error: 
 Accuracy: 47.6%, Avg loss: 1.987495 

Epoch 2
-------------------------------
loss: 1.949969  [    0/50000]
loss: 1.931570  [ 6400/50000]
loss: 2.054480  [12800/50000]
loss: 2.040109  [19200/50000]
loss: 1.910775  [25600/50000]
loss: 1.845672  [32000/50000]
loss: 1.949991  [38400/50000]
loss: 1.962227  [44800/50000]
Test Error: 
 Accuracy: 47.8%, Avg loss: 1.984038 

Epoch 3
-------------------------------
loss: 1.970906  [    0/50000]
loss: 1.881516  [ 6400/50000]
loss: 1.997230  [12800/50000]
loss: 2.003773  [19200/50000]
loss: 2.063114  [25600/50000]
loss: 1.900756  [32000/50000]
loss: 1.978169  [38400/50000]
loss: 2.011283  [44800/50000]
Test Error: 
 Accuracy: 47.9%, Avg loss: 1.982088 

Epoc

In [13]:
epochs = 40
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.920785  [    0/50000]
loss: 1.934054  [ 6400/50000]
loss: 1.952598  [12800/50000]
loss: 1.901371  [19200/50000]
loss: 1.910089  [25600/50000]
loss: 1.883180  [32000/50000]
loss: 1.806921  [38400/50000]
loss: 1.841329  [44800/50000]
Test Error: 
 Accuracy: 51.8%, Avg loss: 1.945338 

Epoch 2
-------------------------------
loss: 1.867405  [    0/50000]
loss: 1.877015  [ 6400/50000]
loss: 1.950830  [12800/50000]
loss: 1.840381  [19200/50000]
loss: 1.930893  [25600/50000]
loss: 1.892443  [32000/50000]
loss: 1.808964  [38400/50000]
loss: 1.946589  [44800/50000]
Test Error: 
 Accuracy: 52.0%, Avg loss: 1.943739 

Epoch 3
-------------------------------
loss: 1.861831  [    0/50000]
loss: 1.983016  [ 6400/50000]
loss: 1.979768  [12800/50000]
loss: 1.833803  [19200/50000]
loss: 1.958081  [25600/50000]
loss: 1.854154  [32000/50000]
loss: 1.936655  [38400/50000]
loss: 2.013726  [44800/50000]
Test Error: 
 Accuracy: 52.3%, Avg loss: 1.942776 

Epoc